In [3]:
import sys
import re
from enum import Enum
from dataclasses import dataclass
from typing import List
import heapq

In [4]:
class CellType(Enum):
    EMPTY = 0
    BLOCK = 1
    SOURCE = 2
    TARGET = 3
    VIA = 4

@dataclass
class Cell:
    kind: CellType
    cost: int = 0

@dataclass
class Coord:
    row: int
    col: int
    layer: int

In [7]:
class MazeRouter:
    NUM_LAYERS = 2

    def __init__(self, in_path: str):
        self.in_path = in_path
        self.min_cost = 0

        self.cols = 0
        self.rows = 0
        self.Direction = 0
        self.via = 0
        self.via_count = 0

        self.grid: List[List[List[Cell]]] = []
        self.nets: List[List[Coord]] = []
        self.same_layer_nets: List[List[Coord]] = []
        self.multi_layer_nets: List[List[Coord]] = []

        self.load_config()

    def load_config(self):
        with open(self.in_path, 'r') as f:
            header = f.readline().strip()
            r, c, b, v = map(int, header.split(','))
            self.rows, self.cols = r, c
            self.Direction, self.via = b, v

            self.grid = [
                [
                    [Cell(CellType.EMPTY) for j in range(self.cols)]
                    for i in range(self.rows)
                ]
                for layer_idx in range(self.NUM_LAYERS)
            ]

            for line in f:
                line = line.strip()
                if line.startswith("OBS"):
                    self.add_obstacle(line)
                elif line.startswith("net"):
                    self.add_net(line)

    def add_obstacle(self, text: str):
        m = re.match(r"OBS\s*\((\d+),\s*(\d+),\s*(\d+)\)", text)
        if not m:
            print("Incorrect Format.", file=sys.stderr)
            return
        layer, row, col = map(int, m.groups())
        if 0 <= layer < self.NUM_LAYERS and 0 <= row < self.rows and 0 <= col < self.cols:
            self.grid[layer][row][col].kind = CellType.BLOCK
        else:
            print("Obstacle out of Range", file=sys.stderr)

    def add_net(self, text: str):
        path: List[Coord] = []
        for m in re.finditer(r"\((\d+),\s*(\d+),\s*(\d+)\)", text):
            layer, row, col = map(int, m.groups())
            if 0 <= layer < self.NUM_LAYERS and 0 <= row < self.rows and 0 <= col < self.cols:
                path.append(Coord(row, col, layer))
            else:
                print("Coordinates out of Range", file=sys.stderr)
                return
        self.nets.append(path)

    def set_SourceAndTarget(self, net_coordinates: List[Coord]):
        for idx, coord in enumerate(net_coordinates):
            kind = CellType.SOURCE if idx == 0 else CellType.TARGET
            self.grid[coord.layer][coord.row][coord.col].kind = kind

    def categorize_nets(self):
        self.same_layer_nets.clear()
        self.multi_layer_nets.clear()
        for idx, net in enumerate(self.nets):
            layers = {coord.layer for coord in net}
            if len(layers) == 1:
                self.same_layer_nets.append(net)
            else:
                self.multi_layer_nets.append(net)

    def back_propagate(self, target: Coord) -> List[Coord]:
        route: List[Coord] = [target]
        current = Coord(target.row, target.col, target.layer)
        via_used = False
        max_iters = self.rows * self.cols * 2
        iters = 0
        while self.grid[current.layer][current.row][current.col].kind != CellType.SOURCE:
            if iters > max_iters:
                print("aborting")
                break
            iters += 1
            cell = self.grid[current.layer][current.row][current.col]
            if cell.kind == CellType.VIA and not via_used:
                via_used = True
                cell.kind = CellType.EMPTY
                current = Coord(current.row, current.col, (current.layer + 1) % self.NUM_LAYERS)
                route.insert(0, current)
                self.via_count += 1
                continue
            best_cost = float('inf')
            best_dir = None
            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                nr, nc = current.row + dr, current.col + dc
                if not (0 <= nr < self.rows and 0 <= nc < self.cols):
                    continue
                neigh = self.grid[current.layer][nr][nc]
                if neigh.kind == CellType.SOURCE:
                    current = Coord(nr, nc, current.layer)
                    route.insert(0, current)
                    return route
                if neigh.kind not in (CellType.BLOCK, CellType.SOURCE) and neigh.cost > 0:
                    if neigh.cost < best_cost:
                        best_cost = neigh.cost
                        best_dir = (nr, nc)
            if best_dir:
                current = Coord(best_dir[0], best_dir[1], current.layer)
                route.insert(0, current)
            else:
                break
            
        return route

    def _lee_wave(self, src: Coord, tgt: Coord, L: int) -> bool:
        INF = float('inf')
        for row_idx in range(self.rows):
            for col_idx in range(self.cols):
                cell = self.grid[L][row_idx][col_idx]
                cell.cost = -1 if cell.kind == CellType.BLOCK else INF

        self.grid[L][src.row][src.col].cost = 0
        pq = [(0, src.row, src.col)]

        while pq:
            acc, row_idx, col_idx = heapq.heappop(pq)
            if acc > self.grid[L][row_idx][col_idx].cost:
                continue
                
            if (row_idx, col_idx) == (tgt.row, tgt.col):
                min_cost = self.grid[L][row_idx][col_idx].cost
                self.min_cost = min_cost
                return True

            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nr, nc = row_idx + dr, col_idx + dc
                if not (0 <= nr < self.rows and 0 <= nc < self.cols):
                    continue

                neigh = self.grid[L][nr][nc]
                if neigh.kind == CellType.BLOCK:
                    continue

                step = 1
                if not ((dr == 0 and L == 0) or (dc == 0 and L == 1)):
                    step = self.Direction

                new_cost = acc + step
                if new_cost < neigh.cost:
                    neigh.cost = new_cost
                    heapq.heappush(pq, (new_cost, nr, nc))

        return False

In [9]:
def main():
    router = MazeRouter('input.txt')

    print(f"Grid: {router.rows} rows × {router.cols} cols, "
          f"Direction Penalty={router.Direction}, via cost={router.via}\n")

    print("Obstacles:")
    for layer in range(router.NUM_LAYERS):
        for r in range(router.rows):
            for c in range(router.cols):
                if router.grid[layer][r][c].kind == CellType.BLOCK:
                    print(f"  layer {layer}, row {r}, col {c}")
    print()

    print("Loaded nets:")
    for idx, net in enumerate(router.nets):
        pts = [(c.layer, c.row, c.col) for c in net]
        print(f"  net{idx}: {pts}")
    print()

    for net in router.nets:
        router.set_SourceAndTarget(net)

    router.categorize_nets()

    print("\n=== ROUTING RESULT ===")
    coords = []
    if router.same_layer_nets:
        src, tgt = router.same_layer_nets[0][0], router.same_layer_nets[0][1]
        if router._lee_wave(src, tgt, L=0):
            path = router.back_propagate(tgt)
            coords = [(c.layer, c.row, c.col) for c in path]
            print("\ncoords for net:", coords)
            print("Total cost:", router.min_cost)
        else:
            print("No Path Found")

if __name__ == "__main__":
    main()


Grid: 8 rows × 8 cols, Direction Penalty=5, via cost=10

Obstacles:
  layer 0, row 0, col 2
  layer 0, row 1, col 2
  layer 0, row 2, col 2
  layer 0, row 3, col 2
  layer 0, row 6, col 3
  layer 0, row 6, col 4
  layer 0, row 6, col 5
  layer 1, row 0, col 2
  layer 1, row 1, col 2
  layer 1, row 2, col 2
  layer 1, row 3, col 2
  layer 1, row 3, col 3
  layer 1, row 3, col 4

Loaded nets:
  net0: [(0, 0, 1), (0, 1, 3)]


=== ROUTING RESULT ===

coords for net: [(0, 0, 1), (0, 1, 1), (0, 2, 1), (0, 3, 1), (0, 4, 1), (0, 4, 2), (0, 4, 3), (0, 3, 3), (0, 2, 3), (0, 1, 3)]
Total cost: 37
